In [3]:
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import numpy as np
from typing import Any
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_folder = '../../data/FMNIST'
fmnist = datasets.FashionMNIST(data_folder, train=True, download=True)
tr_images = fmnist.data
tr_targets = fmnist.targets

In [4]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y) -> None:
        x = x.float()
        x = x.view(-1, 28*28)       # the size -1 is inferred from other dimensions
        self.x, self.y = x,y
    def __getitem__(self, ix) -> Any:
        x,y =self.x[ix], self.y[ix]
        return x.to(device), y.to(device)
    def __len__(self):
        return len(self.x)

In [19]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True)
    return trn_dl

def get_model():
    model = nn.Sequential(
        nn.Linear(28*28, 1000),
        nn.ReLU(),
        nn.Linear(1000, 10)
    )
    loss_func = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=1e-2)
    return model, loss_func, optimizer

In [20]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    opt.step()
    opt.zero_grad()
    return batch_loss.item()

In [21]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [22]:
trn_dl = get_data()
model, loss_fn, optimizer = get_model()

In [ ]:
losses, accuracies = [], []
for epoch in range(5):
    print(epoch)
    epoch_losses, epoch_accuracies = [], []
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        epoch_losses.append(batch_loss)
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)

##### Testing

In [38]:
for _, i in enumerate(iter(trn_dl)):
    print(i)
    print(len(i))
    print(i[0].shape)
    print(i[1].shape)
    x,y = i
    print(x, y)
    break

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([8, 6, 8, 6, 1, 2, 5, 6, 5, 4, 0, 2, 1, 9, 3, 9, 3, 1, 6, 8, 4, 8, 0, 3,
        4, 4, 8, 8, 8, 6, 5, 7], device='cuda:0')]
2
torch.Size([32, 784])
torch.Size([32])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0') tensor([8, 6, 8, 6, 1, 2, 5, 6, 5, 4, 0, 2, 1, 9, 3, 9, 3, 1, 6, 8, 4, 8, 0, 3,
        4, 4, 8, 8, 8, 6, 5, 7], device='cuda:0')


In [30]:
train = FMNISTDataset(tr_images, tr_targets)
len(train)

60000

In [32]:
tr_images.view(-1, 28*28).shape

torch.Size([60000, 784])

In [41]:
print(type(trn_dl))
print(type(iter(trn_dl)))

<class 'torch.utils.data.dataloader.DataLoader'>
<class 'torch.utils.data.dataloader._SingleProcessDataLoaderIter'>
